In [7]:
from skimage.color import rgb2gray, rgb2hsv
from matplotlib.pyplot import bar
from skimage.exposure import histogram
from skimage.util import random_noise
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.filters import median
from skimage.filters import gaussian
from skimage.color import rgb2gray,rgb2hsv

# Show the figures / plots inside the notebook
%matplotlib inline


In [8]:

def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


In [9]:

# import the opencv library
import cv2


# define a video capture object
vid = cv2.VideoCapture(0)

while (True):

    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    dim = (1000, 800)
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    # Display the resulting frame
    frame = cv2.flip(frame, 1)
   
    print(frame.shape)
    border=frame[500:990, 400:790]
    cv2.rectangle(frame,(500,400),(990,790),(0, 0, 0),2)
    hsv = cv2.cvtColor(border, cv2.COLOR_BGR2HSV)
    # define range of skin color in HSV
    # lower_skin = np.array([0,20,70], dtype=np.uint8)
    # upper_skin = np.array([20,255,255], dtype=np.uint8)
    lower_skin = np.array([0,15,0], dtype=np.uint8)
    upper_skin = np.array([17,170,255], dtype=np.uint8)
        
     #extract skin colur imagw  
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    kernel = np.ones((3,3),np.uint8)
   
        
   
        
    #extrapolate the hand to fill dark spots within
    mask = cv2.dilate(mask,kernel,iterations = 5)
        
    #blur the image
    mask = cv2.GaussianBlur(mask,(5,5),100) 
    cv2.imshow('mask',mask)

            
            
       
    
   
    cv2.imshow('frame',frame)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()


(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000, 3)
(800, 1000